In [40]:
# Define a list to store the extracted triples

def parse_triples(grammar):
    triples = []
    # Open the file and process its contents
    with open(f'real\\{grammar}', 'r') as file:
        read_mode = -1

        current_entry = ["", "", ""]

        for line in file:
            if line.startswith("IN:"):
                if not read_mode == "START":
                    triples.append(current_entry)
                read_mode = 0
                current_entry = ["","",""]
            elif line.startswith("OUT:"):
                read_mode = 1
            elif line.startswith("TRUE:"):
                read_mode = 2
            elif line.startswith("Batch") or read_mode == -1:
                continue
            else:
                current_entry[read_mode] += line
    return triples

models_bpe= ["BART_BPE.log", "SIMPLE_6L_BPE.log", "SIMPLE_3L_BPE.log",
            "SIMPLE_1L_BPE.log", "BART_BPE.log"]
models = ["BART_WORDS.log", "SIMPLE_6L_WORDS.log", "SIMPLE_3L_WORDS.log", "SIMPLE_1L_WORDS.log",
          "LSTM_WORDS.log"]




data = {}
for g in models:
    data[g] = parse_triples(g)

In [41]:
print(data[models[0]][1])

['\t<s> die flaschen fallen und die bÃ¤ren fliegen und die teller, deren mutter singt, stehen und die flaschen fliegen und die mutter spielt </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [42]:
import math
import difflib
import numpy as np




def compare_trees(x, y):
    differ = difflib.Differ()
    diff = list(differ.compare(x, y))

    # Process the differences and get indices for additions, deletions, and changes
    added_indices = set()
    deleted_indices = set()
    changed_indices = set()

    index1 = 0
    index2 = 0

    for line in diff:
        if line.startswith('- '):
            deleted_indices.add(index1)
            index1 += 1
        elif line.startswith('+ '):
            added_indices.add(index2)
            index2 += 1
        elif line.startswith(' '):
            index1 += 1
            index2 += 1
        else:
            changed_indices.append((index1, index2))
            index1 += 1
            index2 += 1
    added_deleted = added_indices.intersection(deleted_indices)
    added_indices -= added_deleted
    deleted_indices -= added_deleted
    changed_indices = changed_indices.union(added_deleted)

    return (added_indices, deleted_indices, changed_indices)

def direct_compare(x,y):
    assert len(x) == len(y), f"{x} != {y}"
    changes = []
    for i, e in enumerate(x):
        if e != y[i]:
            changes.append((i, e, y[i]))
    return changes

def is_open(s: str):
    return s.startswith("(")

def is_closed(s: str):
    return s.startswith(")")

def is_word(s: str):
    return not is_open(s) and not is_closed(s)
def tree_height(tree, idx):
    length = len(tree)
    s = tree[idx]
    if is_word(s):
        return 0
    if is_open(s):
        dir = 1
        _is_open = lambda x: is_open(x)
        _is_closed = lambda x: is_closed(x)
    else:
        dir = -1
        _is_open = lambda x: is_closed(x)
        _is_closed = lambda x: is_open(x)

    unclosed = 1
    max_val = 1
    i = idx + dir
    while unclosed != 0 and 0 <= i < length:
        s = tree[i]
        if _is_open(s):
            unclosed += 1
            max_val = max(unclosed, max_val)
        elif _is_closed(s):
            unclosed -= 1
        i += dir

    return max_val

def tree_min_height(tree, idx):
    length = len(tree)
    s = tree[idx]
    if is_word(s):
        return 0
    if is_open(s):
        dir = 1
        _is_open = lambda x: is_open(x)
        _is_closed = lambda x: is_closed(x)
    else:
        dir = -1
        _is_open = lambda x: is_closed(x)
        _is_closed = lambda x: is_open(x)

    unclosed = 1
    min_val = 15
    i = idx + dir
    while unclosed != 0 and 0 <= i < length:
        s = tree[i]
        if _is_open(s):
            unclosed += 1
        elif _is_closed(s):
            min_val = min(unclosed, min_val)
            unclosed -= 1
        i += dir

    return min_val

def stat_changes(label, changes):
    ids = [change[0] for change in changes]
    depths = []
    heights = []


    bracket_type_count = {}

    def count_type(type):
        if type in bracket_type_count:
            bracket_type_count [type] += 1
        else:
            bracket_type_count[type] = 1

    ids.sort()

    unclosed = 0
    for i, e in enumerate(label):
        if is_open(e):
            unclosed += 1
        elif is_closed(e):
            unclosed -= 1
        if i in ids:
            depths.append(unclosed)


    for i in ids:
        # heights.append(next_word(i))
        height = tree_height(label, i)
        heights.append(height)
        if height > 0:
            count_type(label[i])


    return (depths, heights, bracket_type_count)

def count_bin(cap, list):
    bins = np.zeros(cap)
    for x in list:
        if x >= cap:
            bins[cap-1]+=1
        elif 0 <= x:
            bins[x]+=1
    return bins

def compare_triple(triple, return_all=False):
    inp, out, label = triple
    label = label.split()[:-1]

    length = min(len(out), len(label))
    if length == 0 or not is_open(label[0]):
        return None

    out = out.split()[:len(label)]
    label = label[:len(out)]

    if not return_all:
        changes = direct_compare(label, out)
    else:
        changes = [(idx, None) for idx, l in enumerate(label)]
    if not changes:
        return None

    stats = stat_changes(label, changes)
    depths, heights, per_bracket = stats
    depths = count_bin(15, depths)
    heights = count_bin(20, heights)

    return changes, depths, heights, per_bracket

def get_d(bin):
    d = []
    for i in range(6):
        d.append(np.sum(bin[i:]))
    return d

def stack_bins(bins):
    stacked = []
    for e in bins:
        stacked.append(e)
    return np.sum(np.array(stacked), axis=0)

def collate_dicts(dicts):
    collated = {}
    for dict in dicts:
        for k, v in dict.items():
            collated[k] = collated.get(k, 0) + v
    return collated

def compute(model):
    triples = data[model]
    all = [compare_triple(triple, return_all=True) for triple in triples]
    all = [s for s in all if s]

    errors = [compare_triple(triple) for triple in triples]
    errors = [s for s in errors if s]

    depths = stack_bins([e[1] for e in errors])
    heights = stack_bins([e[2] for e in errors])
    bracket_counts = collate_dicts([e[3] for e in errors])

    all_depths = stack_bins([e[1] for e in all])
    all_heights = stack_bins([e[2] for e in all])
    all_bracket_counts = collate_dicts([e[3] for e in errors])


    return (heights, all_heights), (depths, all_depths), (bracket_counts, all_bracket_counts)


computed = {}
for m in models:
    computed[m] = compute(m)

In [44]:
from tabulate import tabulate

for model in models:
    ((h, ah), (d, ad), (br, abr)) = computed[model]

    heights = np.vstack((h, ah)).T
    depths = np.vstack((d, ad)).T

    print(model)
    for i, e in enumerate(zip(h, ah)):
        print(str(e[0]) + " \t " + str(e[1]))

    print("Brackets")
    print(br)
    print("--all--")
    print(abr)
    print("----")
    print("")

def residual():

    ((heights, all_heights), (depths, all_depths)) = computed[models[0]]
    total_errs = np.sum(heights)
    total = np.sum(all_heights)

    print("Total errs:")
    print(total_errs)


    print("Out:")
    print(len(heights))
    print(len(all_heights))
    print(heights)
    print(all_heights)
    print(heights / all_heights)

    print("Out 2:")
    print(len(depths))
    print(len(all_depths))
    print(depths)
    print(all_depths)
    print(depths / all_depths)


    d_heights = get_d(heights)
    print("___")
    print(d_heights)
    d_heights = np.array(d_heights) / total_errs

    occs = [1.0, 0.544, 0.166, 0.072, 0.03, 0.01]
    occs_2DCL = [1.0, 0.545, 0.175, 0.075, 0.031, 0.01]
    occ_num = len(occs)

    print(d_heights)
    print((d_heights / occs))

BART_WORDS.log
2549.0 	 333368.0
5875.0 	 811462.0
2807.0 	 116510.0
1788.0 	 79178.0
426.0 	 47150.0
434.0 	 31298.0
402.0 	 19652.0
189.0 	 11264.0
79.0 	 2958.0
58.0 	 1060.0
26.0 	 364.0
14.0 	 124.0
7.0 	 50.0
3.0 	 14.0
2.0 	 4.0
0.0 	 0.0
0.0 	 0.0
0.0 	 0.0
0.0 	 0.0
0.0 	 0.0
Brackets
{'(d': 2011, '(s': 4082, '(n_s': 530, ')n_s': 62, '(komma,': 243, ')komma': 96, '(deren': 330, ')deren': 49, '(t': 49, '(n_p': 825, ')n_p': 67, '(v_p': 534, ')v_p': 68, ')t': 53, '(v_s': 305, ')v_s': 61, ')s': 1002, '(und': 577, ')und': 53, ')d': 64, '(p': 682, ')p': 367}
--all--
{'(d': 2011, '(s': 4082, '(n_s': 530, ')n_s': 62, '(komma,': 243, ')komma': 96, '(deren': 330, ')deren': 49, '(t': 49, '(n_p': 825, ')n_p': 67, '(v_p': 534, ')v_p': 68, ')t': 53, '(v_s': 305, ')v_s': 61, ')s': 1002, '(und': 577, ')und': 53, ')d': 64, '(p': 682, ')p': 367}
----

SIMPLE_6L_WORDS.log
82584.0 	 340292.0
52106.0 	 826659.0
5997.0 	 118859.0
4933.0 	 80012.0
718.0 	 47848.0
119.0 	 32134.0
237.0 	 19954.0
334.

In [51]:
import os
import json
import re
file_loc = r"C:\Users\MKhal\OneDrive\Desktop\UNI\Bachelorarbeit\DEV\server-export\data\2D-Bracket_420.json"
token_pattern = r"\(\w*|\)|[^()]+"

set = file_loc

with open (os.path.join(file_loc, set), 'r') as f:
    file = json.load(f)

data = [e["l"] for e in file["data"]]

all = []

for d in data:
    tokens = re.findall(pattern=token_pattern, string=d)
    for i in range(len(tokens)):
        all.append(tree_height(tokens, i))

all_bin = count_bin(15, all)
print(all_bin)

KeyboardInterrupt: 

In [ ]:
def is_right(e):
    _, x, y = e

    if is_open(x):
        return is_open(y)
    if is_closed(x):
        return is_closed(y)
    return False


all_tok = []
for s in all:
    all_tok.extend(s[0])


print(len(all_tok))
print(len([s for s in all_tok if not is_right(s)]))